**гибридный поиск версия 1.4:**

- замена эмбеддера ~`rubert_tiny2`~ от Давида Дале на `rubert_base` от ai-forever

### **data prepairing**

In [ ]:
%%capture
!pip install openpyxl

In [ ]:
import pandas as pd

In [ ]:
df_req_cer = pd.read_excel('ВК_Реквизиты_Справки.xlsx')
df_doc_pen = pd.read_excel('ВК_Документ_Пенсия.xlsx')

assert len(df_req_cer.columns.tolist()) == len(df_doc_pen.columns.tolist()), "different num of columns"
assert df_req_cer.columns.tolist() == df_doc_pen.columns.tolist(), "different columns"

df = pd.concat([df_doc_pen, df_req_cer], ignore_index=True)
df

,Краткое наименование услуги (суть),Кодирование в АИС МФЦ / СЭО,Заявитель в МФЦ,Способ подачи документов,Способ выдачи результата услуги,Пакет документов для отправки в Военкомат при личном обращении:,Пакет документов для отправки в Военкомат при обращении доверенного лица дополнительно предоставляют:,Срок предоставления услуги,Результат предоставления услуги,Возможность совместной подачи
0,Удостоверение инвалида,ДОКУМЕНТ01,"Гражданин РФ, являющийся инвалидом вследствии ...",1. Лично в МФЦ\n2. Лично в ОИВ,1. Лично в МФЦ\n2. Лично в ОИВ (в случае подач...,1. Заявление. \n2. Паспорт гражданина РФ (копи...,1. Паспорт доверенного лица (копия). \n2. Копи...,30 календарных дней,1. Удостоверение «Инвалида о праве на льготы»....,ВЫПЛАТЫ05\nПЕНСИЯ03
1,Дубликат удостоверения инвалида,ДОКУМЕНТ01,"Гражданин РФ, являющийся инвалидом вследствии ...",1. Лично в МФЦ\n2. Лично в ОИВ,1. Лично в МФЦ\n2. Лично в ОИВ (в случае подач...,1. Заявление. \n2. Паспорт гражданина РФ (копи...,1. Паспорт доверенного лица (копия). \n2. Копи...,30 календарных дней,1. Дубликат удостоверения «Инвалида о праве на...,ВЫПЛАТЫ05\nПЕНСИЯ03
2,Дубликат пенсионного удостоверения от Военкомата,ДОКУМЕНТ02,"Гражданин РФ, состоящий на пенсионном обеспече...",1. Лично в МФЦ\n2. Лично в ОИВ,1. Лично в МФЦ\n2. Лично в ОИВ (в случае подач...,1. Заявление. \n2. Паспорт РФ (копия). \n3. Фо...,1. Паспорт доверенного лица (копия). \n2. Копи...,30 календарных дней,1. Дубликат пенсионного удостоверения. \n2. Ув...,NaN
3,Удостоверение ветерана военной службы,ДОКУМЕНТ03,"1. Гражданин РФ, состоящий на пенсионном обесп...",1. Лично в МФЦ\n2. Лично в ОИВ,1. Лично в МФЦ\n2. Лично в ОИВ (в случае подач...,1. Заявление (первичное). \n2. Паспорт граждан...,1. Паспорт доверенного лица (копия). \n2. Копи...,21 день для направления документов в Миноборон...,1. Удостоверение «Ветеран военной службы». \n2...,NaN
4,Дубликат удостоверения ветерана военной службы,ДОКУМЕНТ03,"1. Гражданин РФ, состоящий на пенсионном обесп...",1. Лично в МФЦ\n2. Лично в ОИВ,1. Лично в МФЦ\n2. Лично в ОИВ (в случае подач...,1. Заявление (дубликат). \n2. Паспорт граждани...,1. Паспорт доверенного лица (копия). \n2. Копи...,21 день для направления документов в Миноборон...,1. Дубликат удостоверения «Ветеран военной слу...,NaN
...,...,...,...,...,...,...,...,...,...,...
151,Выдача справок о праве на получение компенсаци...,СПРАВКА10,"1. Члены семей военнослужащих, погибших (умерш...",Лично в МФЦ,Лично в МФЦ,1. Заявление. 2. Паспорт РФ (копия). 3. Свидет...,1. Паспорт доверенного лица (копия). 2. Копия ...,30 календарных дней,1. Справка. 2. Уведомление об отказе в выдаче ...,NaN
152,Справка о праве на получение компенсаций расхо...,СПРАВКА10,"1. Члены семей военнослужащих, погибших (умерш...",Лично в МФЦ,Лично в МФЦ,1. Заявление. 2. Паспорт РФ (копия). 3. Свидет...,1. Паспорт доверенного лица (копия). 2. Копия ...,30 календарных дней,1. Справка. 2. Уведомление об отказе в выдаче ...,NaN
153,Справка о праве на получение компенсационных в...,СПРАВКА10,"1. Члены семей военнослужащих, погибших (умерш...",Лично в МФЦ,Лично в МФЦ,1. Заявление. 2. Паспорт РФ (копия). 3. Свидет...,1. Паспорт доверенного лица (копия). 2. Копия ...,30 календарных дней,1. Справка. 2. Уведомление об отказе в выдаче ...,NaN
154,Выдача справок о праве на получение компенсаци...,СПРАВКА10,"1. Члены семей военнослужащих, погибших (умерш...",Лично в МФЦ,Лично в МФЦ,1. Заявление. 2. Паспорт РФ (копия). 3. Свидет...,1. Паспорт доверенного лица (копия). 2. Копия ...,30 календарных дней,1. Справка. 2. Уведомление об отказе в выдаче ...,NaN


**вытаскиваем колонку, которая будет векторизоваться и по которой будет производиться поиск**

In [ ]:
services = df['Краткое наименование услуги (суть)']
services_list = services.tolist()
services

0                                 Удостоверение инвалида
1                        Дубликат удостоверения инвалида
2       Дубликат пенсионного удостоверения от Военкомата
3                  Удостоверение ветерана военной службы
4         Дубликат удостоверения ветерана военной службы
                             ...                        
151    Выдача справок о праве на получение компенсаци...
152    Справка о праве на получение компенсаций расхо...
153    Справка о праве на получение компенсационных в...
154    Выдача справок о праве на получение компенсаци...
155    Выдача справок о праве на получение компенсаци...
Name: Краткое наименование услуги (суть), Length: 156, dtype: object

### **`BM25`-part**

- [BM25](https://github.com/dorianbrown/rank_bm25)
- Note that this package doesn't do any text preprocessing. If you want to do things like lowercasing, stopword removal, stemming, etc, you need to do it yourself.
- The only requirements is that the class receives a list of lists of strings, which are the document tokens.
- Good to note that we also need to tokenize our query, and apply the same preprocessing steps we did to the documents in order to have an apples-to-apples comparison

In [ ]:
%%capture
!pip install rank_bm25
!pip install pymystem3
!pip install pymorphy2==0.8

In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi

In [ ]:
import re
import pymorphy2

# стоп-слова грузим отдельно
with open('/home/darhanovev/stopwords/russian.txt', 'r') as file:
    stop_words = {word.strip() for word in file}

lemmatizer = pymorphy2.MorphAnalyzer()

In [ ]:
TOKEN_WORD = re.compile('[а-яА-ЯёЁ]{2,}')


def remove_lemmatize_preprocessing(sentence: str, words_to_remove: set) -> list:
    """ приводит к нижнему регистру, убирает числа и мусорные слова, лемматизирует """
    if isinstance(sentence, str):
        regex_tmp = re.findall(TOKEN_WORD, sentence.lower())
        remove_tmp = [lemmatizer.parse(token.strip())[0].normal_form for token in regex_tmp if token not in words_to_remove]
        return remove_tmp
    return []


def one_sentence_preprocessing(sentence: str, words_to_remove: set) -> str:
    """ применяет препроцессинги к одному предложению """
    sentence_words = remove_lemmatize_preprocessing(sentence, words_to_remove)
    result = ' '.join(sentence_words)
    return result

In [ ]:
services_tokenized = [one_sentence_preprocessing(service, stop_words).split() for service in services_list]
services_tokenized[:10]

[['удостоверение', 'инвалид'],
 ['дубликат', 'удостоверение', 'инвалид'],
 ['дубликат', 'пенсионный', 'удостоверение', 'военкомат'],
 ['удостоверение', 'ветеран', 'военный', 'служба'],
 ['дубликат', 'удостоверение', 'ветеран', 'военный', 'служба'],
 ['удостоверение', 'ветеран', 'боев', 'действие'],
 ['дубликат', 'удостоверение', 'ветеран', 'боев', 'действие'],
 ['удостоверение', 'член', 'семья', 'умерший', 'ветеран', 'боев', 'действие'],
 ['дубликат',
  'удостоверение',
  'член',
  'семья',
  'умерший',
  'ветеран',
  'боев',
  'действие'],
 ['продление', 'выплата', 'пенсия', 'случай', 'потеря', 'кормилец']]

In [ ]:
bm25_vk = BM25Okapi(services_tokenized)


def make_query_bm_25(query: str,
                     top: int = 5,
                     corpus=services,
                     words_to_remove: set = stop_words):
    """
    возвращает:
        - [{'corpus_id': id, 'score': score}, ...]
        - score - значения Okapi BM25 в диапазоне [0, 1]
    """

    query_cleaned = one_sentence_preprocessing(query, words_to_remove)
    query_tokenized = query_cleaned.split()

    scores = bm25_vk.get_scores(query_tokenized)
    scores /= np.max(scores)

    top_k = np.argsort(scores)[::-1][:top]
    result = [{'corpus_id': i, 'score': scores[i]} for i in top_k]
    # result = [corpus[i] for i in top_k]

    return result

In [ ]:
scores = make_query_bm_25('Дубликат пенсионного удостоверения от Военкомата')
scores

[{'corpus_id': 2, 'score': 1.0},
 {'corpus_id': 1, 'score': 0.43077383354725285},
 {'corpus_id': 6, 'score': 0.3645770993163829},
 {'corpus_id': 4, 'score': 0.3645770993163829},
 {'corpus_id': 8, 'score': 0.2962827367493113}]

### **cosine similarity part**

- `Sentence Transformers`
- эмбеддер: `ruBERT-base`
- [Hugging Face](https://huggingface.co/ai-forever/ruBert-base)

In [ ]:
%%capture
!pip install torch
!pip install transformers
!pip install sentence_transformers

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
rubert_base_path = "/home/darhanovev/hugging_face/rubert_base"

model_cosine = SentenceTransformer(
    model_name_or_path=rubert_base_path,
    device=device,
    local_files_only=True
)

No sentence-transformers model found with name /home/darhanovev/hugging_face/rubert_base. Creating a new one with mean pooling.


In [ ]:
services_emb_cos = model_cosine.encode(services_list)


def make_query_cosine(query: str,
                      top: int = 5,
                      model=model_cosine,
                      embeddings=services_emb_cos):
    """
    возвращает:
        - [{'corpus_id': id, 'score': score}, ...]
        - score - нормированная на диапазон [0, 1] косинусная близость
    """

    query_emb = model.encode(query)
    result = util.semantic_search(query_emb, embeddings)[0][:top]
    for dct in result:
        dct['score'] = (dct['score'] + 1) / 2

    return result

In [ ]:
result = make_query_cosine('Дубликат пенсионного удостоверения от Военкомата')
result

[{'corpus_id': 2, 'score': 0.9999999105930328},
 {'corpus_id': 1, 'score': 0.9293651282787323},
 {'corpus_id': 4, 'score': 0.9071397185325623},
 {'corpus_id': 42, 'score': 0.8887702524662018},
 {'corpus_id': 36, 'score': 0.8866200745105743}]

In [ ]:
for dct in result:
    print(dct['score'], services_list[dct['corpus_id']])

0.9999999105930328 Дубликат пенсионного удостоверения от Военкомата
0.9293651282787323 Дубликат удостоверения инвалида
0.9071397185325623 Дубликат удостоверения ветерана военной службы
0.8887702524662018 Предоставление копий из документов воинского учёта
0.8866200745105743 Выдача копий из документов воинского учёта


### **`Bi-Encoder`-part**

- асимметричный поиск
- `Sentence Transformers`
- эмбеддер: `ruBERT`
- [Hugging Face](https://huggingface.co/DiTy/bi-encoder-russian-msmarco)
- [статья с Хабра](https://habr.com/ru/articles/834356/)

In [ ]:
%%capture
!pip install torch
!pip install sentence_transformers

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
bi_encoder_path = "/home/darhanovev/hugging_face/bi_encoder_russian_msmarco"

model_bi = SentenceTransformer(
    model_name_or_path=bi_encoder_path,
    device=device,
    local_files_only=True
)

In [ ]:
services_emb_bi = model_bi.encode(services_list)


def make_query_asymm(query: str,
                     top: int = 5,
                     model=model_bi,
                     embeddings=services_emb_bi):
    """
    возвращает:
        - [{'corpus_id': id, 'score': score}, ...]
        - score - нормированная на диапазон [0, 1] косинусная близость
    """

    query_emb = model.encode(query)
    result = util.semantic_search(query_emb, embeddings)[0][:top]
    for dct in result:
        dct['score'] = (dct['score'] + 1) / 2

    return result

In [ ]:
result = make_query_asymm('Дубликат пенсионного удостоверения от Военкомата')
result

[{'corpus_id': 2, 'score': 1.0},
 {'corpus_id': 4, 'score': 0.7623136043548584},
 {'corpus_id': 1, 'score': 0.7546685338020325},
 {'corpus_id': 6, 'score': 0.7182275950908661},
 {'corpus_id': 8, 'score': 0.6954275220632553}]

In [ ]:
for dct in result:
    print(dct['score'], services_list[dct['corpus_id']])

1.0 Дубликат пенсионного удостоверения от Военкомата
0.7623136639595032 Дубликат удостоверения ветерана военной службы
0.754668653011322 Дубликат удостоверения инвалида
0.7182276248931885 Дубликат удостоверения ветерана боевых действий
0.6954275667667389 Дубликат удостоверения члену семьи умершего ветерана боевых действий


### **unite them**

In [ ]:
def scores_merging(
    *args,
    top: int,
    pooling: bool = False,
    threshold: float | None = None,
    dataframe=df
) -> list[tuple[int | float]]:

    """
    объединяет результаты произвольного количества поисковиков
    принимает: несколько словарей {id: score, ...}, {id: score, ...}; id - DataFrame index
    оставляет уникальные ключи, суммирует скоры
    возвращает: список кортежей [(id, score), ...], [(id, score), ...]; id - DataFrame index

    if pooling is True:
        делает: Взвешенная сумма + Max pooling + топ-N
        возвращает: список кортежей [(id, score), ...], [(id, score), ...]; id - код услуги

    if threshold is not None:
        возвращает: список кортежей [(id, score), ...], [(id, score), ...]; score > threshold
    """

    tmp_result = {}

    # суммируем скоры с коэффициентами
    for arg in args:
        for key, value in arg.items():
            if key in tmp_result:
                tmp_result[key] += value
            else:
                tmp_result[key] = value

    if pooling:
        # получаем словарь вида {'ДОКУМЕНТ01': score, 'СПРАВКА01': score, ...}
        result = {}
        for key, value in tmp_result.items():
            new_key = dataframe.iloc[key, 1]
            if new_key in result:
                result[new_key] = max(result[new_key], value)
            else:
                result[new_key] = value
        tmp_result = result

    # сортируем по итоговому значению скора и берём топ
    result = sorted(tmp_result.items(), key=lambda item: -item[1])[:top]

    if threshold is not None:
        result = list(filter(lambda elem: elem[1] > threshold, result))

    return result

In [ ]:
def hybrid_search(
    query: str,
    top: int = 5,
    pooling: bool = False,
    threshold: float | None = None,
    weights={'bm_25': 0.333, 'cosine': 0.333, 'asymm': 0.333},
    dataframe=df,
    retrieve_bm_25=make_query_bm_25,
    retrieve_cosine=make_query_cosine,
    retrieve_asymm=make_query_asymm
):
    """
    возвращает спискок кортежей вида [(DataFrame_id: int, score: float), ...]
                            или вида [(код услуги: str, score: float), ...]
    """

    # получаем скоры поисковиков, умножаем на веса
    scores_bm_25 = { dct['corpus_id']: dct['score'] * weights['bm_25'] for dct in make_query_bm_25(query, top) }
    scores_cosine = { dct['corpus_id']: dct['score'] * weights['cosine'] for dct in make_query_cosine(query, top) }
    scores_asymm = { dct['corpus_id']: dct['score'] * weights['asymm'] for dct in make_query_asymm(query, top) }

    # объединяем результаты в требуемом формате
    result = scores_merging(scores_bm_25, scores_cosine, scores_asymm, top=top, pooling=pooling, threshold=threshold, dataframe=df)

    return result

In [ ]:
%%time
hybrid_search('Дубликат пенсионного удостоверения от Военкомата', pooling=False)

CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.12 s


[(2, 0.9990000198483466),
 (1, 0.7041395936492629),
 (4, 0.6782620157130663),
 (6, 0.3605739731617873),
 (8, 0.33023953107084475)]

In [ ]:
%%time
hybrid_search('Дубликат пенсионного удостоверения от Военкомата', pooling=False, threshold=0.6)

CPU times: user 1.73 s, sys: 9.27 ms, total: 1.74 s
Wall time: 1.7 s


[(2, 0.9990000198483466), (1, 0.7041395936492629), (4, 0.6782620157130663)]

In [ ]:
%%time
hybrid_search('Дубликат пенсионного удостоверения от Военкомата', pooling=True)

CPU times: user 1.14 s, sys: 0 ns, total: 1.14 s
Wall time: 1.01 s


[('ДОКУМЕНТ02', 0.9990000198483466),
 ('ДОКУМЕНТ01', 0.7041395936492629),
 ('ДОКУМЕНТ03', 0.6782620157130663),
 ('ДОКУМЕНТ04', 0.3605739731617873),
 ('ДОКУМЕНТ05', 0.33023953107084475)]

In [ ]:
%%time
hybrid_search('Дубликат пенсионного удостоверения от Военкомата', pooling=True, threshold=0.6)

CPU times: user 1.69 s, sys: 12.6 ms, total: 1.7 s
Wall time: 1.69 s


[('ДОКУМЕНТ02', 0.9990000198483466),
 ('ДОКУМЕНТ01', 0.7041395936492629),
 ('ДОКУМЕНТ03', 0.6782620157130663)]

### **demo**

In [ ]:
print("ДЕМОНСТРАЦИЯ: ПЕРЕМНОЖАЕМ ТЕСТОВЫЕ ВЕСА И СКОРЫ")
weights = {'bm_25': 0.333, 'cosine': 0.333, 'asymm': 0.333}

test_score_1 = [{'corpus_id': 2, 'score': 1.0},
 {'corpus_id': 1, 'score': 0.43077383354725285},
 {'corpus_id': 6, 'score': 0.3645770993163829},
 {'corpus_id': 4, 'score': 0.3645770993163829},
 {'corpus_id': 8, 'score': 0.2962827367493113}]

test_score_2 = [{'corpus_id': 2, 'score': 1.0000000596046448},
 {'corpus_id': 1, 'score': 0.929090827703476},
 {'corpus_id': 4, 'score': 0.9099321067333221},
 {'corpus_id': 11, 'score': 0.9039898812770844},
 {'corpus_id': 65, 'score': 0.9038759469985962}]

test_score_3 = [{'corpus_id': 2, 'score': 1.0},
 {'corpus_id': 4, 'score': 0.762313574552536},
 {'corpus_id': 1, 'score': 0.7546685934066772},
 {'corpus_id': 6, 'score': 0.7182275801897049},
 {'corpus_id': 8, 'score': 0.6954275369644165}]

scores_bm_25_dict = { dct['corpus_id']: dct['score'] * weights['bm_25'] for dct in test_score_1 }
scores_cosine_dict = { dct['corpus_id']: dct['score'] * weights['cosine'] for dct in test_score_2 }
scores_asymm_dict = { dct['corpus_id']: dct['score'] * weights['asymm'] for dct in test_score_3 }

print("INPUT:")
print(test_score_1)
print(test_score_2)
print(test_score_3)
print()
print("OUTPUT:")
print(scores_bm_25_dict)
print(scores_cosine_dict)
print(scores_asymm_dict)

ДЕМОНСТРАЦИЯ: ПЕРЕМНОЖАЕМ ТЕСТОВЫЕ ВЕСА И СКОРЫ
INPUT:
[{'corpus_id': 2, 'score': 1.0}, {'corpus_id': 1, 'score': 0.43077383354725285}, {'corpus_id': 6, 'score': 0.3645770993163829}, {'corpus_id': 4, 'score': 0.3645770993163829}, {'corpus_id': 8, 'score': 0.2962827367493113}]
[{'corpus_id': 2, 'score': 1.0000000596046448}, {'corpus_id': 1, 'score': 0.929090827703476}, {'corpus_id': 4, 'score': 0.9099321067333221}, {'corpus_id': 11, 'score': 0.9039898812770844}, {'corpus_id': 65, 'score': 0.9038759469985962}]
[{'corpus_id': 2, 'score': 1.0}, {'corpus_id': 4, 'score': 0.762313574552536}, {'corpus_id': 1, 'score': 0.7546685934066772}, {'corpus_id': 6, 'score': 0.7182275801897049}, {'corpus_id': 8, 'score': 0.6954275369644165}]

OUTPUT:
{2: 0.333, 1: 0.14344768657123522, 6: 0.1214041740723555, 4: 0.1214041740723555, 8: 0.09866215133752068}
{2: 0.3330000198483467, 1: 0.3093872456252575, 4: 0.3030073915421963, 11: 0.3010286304652691, 65: 0.30099069035053255}
{2: 0.333, 4: 0.25385042032599453

In [ ]:
print("ДЕМОНСТРАЦИЯ: взвешенная сумма + топ-N")
print("INPUT:")
print(scores_bm_25_dict)
print(scores_cosine_dict)
print(scores_asymm_dict)
print()
print("OUTPUT:")
scores_merging(scores_bm_25_dict, scores_cosine_dict, scores_asymm_dict, top=5, threshold=0.7)

ДЕМОНСТРАЦИЯ: взвешенная сумма + топ-N
INPUT:
{2: 0.333, 1: 0.14344768657123522, 6: 0.1214041740723555, 4: 0.1214041740723555, 8: 0.09866215133752068}
{2: 0.3330000198483467, 1: 0.3093872456252575, 4: 0.3030073915421963, 11: 0.3010286304652691, 65: 0.30099069035053255}
{2: 0.333, 4: 0.25385042032599453, 1: 0.25130464160442356, 6: 0.23916978420317175, 8: 0.2315773698091507}

OUTPUT:


[(2, 0.9990000198483466), (1, 0.7041395738009163)]

In [ ]:
print("ДЕМОНСТРАЦИЯ: взвешенная сумма + maxpooling + топ-N")
print("INPUT:")
print(scores_bm_25_dict)
print(scores_cosine_dict)
print(scores_asymm_dict)
print()
print("OUTPUT:")
scores_merging(scores_bm_25_dict, scores_cosine_dict, scores_asymm_dict, top=5, pooling=True, threshold=0.7)

ДЕМОНСТРАЦИЯ: взвешенная сумма + maxpooling + топ-N
INPUT:
{2: 0.333, 1: 0.14344768657123522, 6: 0.1214041740723555, 4: 0.1214041740723555, 8: 0.09866215133752068}
{2: 0.3330000198483467, 1: 0.3093872456252575, 4: 0.3030073915421963, 11: 0.3010286304652691, 65: 0.30099069035053255}
{2: 0.333, 4: 0.25385042032599453, 1: 0.25130464160442356, 6: 0.23916978420317175, 8: 0.2315773698091507}

OUTPUT:


[('ДОКУМЕНТ02', 0.9990000198483466), ('ДОКУМЕНТ01', 0.7041395738009163)]